In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb

## NOTE: this model is gonna be slightly better, but suspiscious with non-rolling window of data


# reading dat
dat = pd.read_csv("MLDataSplit/combined.csv")

# modifying dat to remove character values
dat["WinAsInt"] = dat["Win"].transform(lambda x: int(x == "WIN"))
dat = dat.drop(axis = 1, labels = "Win")
print(dat.shape)

In [ ]:
correlations = dat.corr()["WinAsInt"].sort_values(ascending = False)

print(correlations["SUP_Deaths"])

In [ ]:
print(correlations[correlations < -0.03])
print()
print(correlations[correlations > 0.12])

In [ ]:
input_vars = correlations[correlations < -0.03].index.to_list() + correlations[correlations > 0.12].index.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

In [ ]:
param = {'max_depth': 3, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 500
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
xgb.plot_importance(bst)

In [ ]:
prediction_data = pd.read_csv("MLDataSplit/LEC_2023_Spring_Test.csv")
teams = prediction_data["Team"].to_frame()
prediction_data = prediction_data.drop(axis = 1, labels = ["Team"])
input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Small Model Scores"] = scores

print(teams)

In [ ]:
input_vars = dat.columns.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
# print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

In [ ]:
param = {'max_depth': 3, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 500
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
xgb.plot_importance(bst)

In [ ]:
prediction_data = pd.read_csv("MLDataSplit/LEC_2023_Spring_Test.csv")
# teams = prediction_data["Team"].to_frame()
prediction_data = prediction_data.drop(axis = 1, labels = ["Team"])
input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Full Model Scores"] = scores

print(teams)

In [ ]:
matchups = pd.DataFrame()

for i in range(30):
    blue_dat = teams.loc[i * 2]
    red_dat = teams.loc[i * 2 + 1]
    blue_team = blue_dat["Team"]
    red_team = red_dat["Team"]

    # skew a little to blue side win rate
    scores = blue_dat.drop("Team") - red_dat.drop("Team") > -0.02
    newrow = pd.Series([blue_team, red_team, int(scores[0]), int(scores[1])])

    newrow = newrow.to_frame().transpose()
    matchups = pd.concat([matchups, newrow], axis = 0)
    
matchups.columns = ["B_Team", "R_Team", "Small Model", "Full Model"]
print(matchups)

    
matchups.to_csv("predictions/splitModel.csv")    

    
